# ChatClovaX

This notebook provides a quick overview for getting started with Naver’s CLOVA [chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) via CLOVA Studio. For detailed documentation of all ChatClovaX features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/chat_models/langchain_anthropic.chat_models.ChatAnthropic.html).

CLOVA Studio has several chat models. You can find information about latest models and their costs, context windows, and supported input types in the CLOVA Studio API Guide [documentation](https://api.ncloud-docs.com/docs/clovastudio-chatcompletions).

## Overview
### Integration details

| Class | Package | Local | Serializable | [JS support](https://js.langchain.com/v0.2/docs/integrations/chat/naver) | Package downloads | Package latest |
| :--- | :--- |:-----:| :---: |:------------------------------------------------------------------------:| :---: | :---: |
| [ChatClovaX](https://api.python.langchain.com/en/latest/chat_models/langchain_naver.chat_models.ChatClovaX.html) | [langchain-naver](https://api.python.langchain.com/en/latest/naver_api_reference.html) |   ❌   | beta/❌ |                                    ❌                                     | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-naver?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-naver?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling/) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
|:------------------------------------------:| :---: | :---: | :---: |  :---: | :---: |:-----------------------------------------------------:| :---: |:------------------------------------------------------:|:----------------------------------:|
|❌| ✅ | ✅ | ❌ | ❌ | ❌ |                          ✅                            | ✅ |                           ✅                            |                 ❌                  | 

## Setup

Before using the chat model, you must go through the three steps below.

- Creating NCP account (https://www.ncloud.com/)
- Apply to use CLOVA Studio (https://www.ncloud.com/product/aiService/clovaStudio > ‘Get Started’)
- Find API Key after creating CLOVA Studio testApp or serviceApp (참고: https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#테스트앱생성)

### Credentials

- CLOVA Studio requires 2 keys (`NCP_CLOVASTUDIO_API_KEY` and `NCP_APIGW_API_KEY`).
    - `NCP_CLOVASTUDIO_API_KEY` can issued per serviceApp or testApp
    - `NCP_APIGW_API_KEY` can issued per account
- The two API Keys could be found by clicking `App Request Status` > `Service App, Test App List` > `‘Details’ button for each app` in CLOVA Studio.
    - https://clovastudio.ncloud.com/studio-application/service-app

### Installation

The LangChain Naver integration lives in the `langchain-naver` package:

In [ ]:
# install package
!pip install -qU langchain-naver

## Instantiation

1. example model: HCX-DASH-001
2. parameters: Input with working values from templates (temperature, max_tokens, max_retries)
3. additional parameters: Input parameters provided only in CLOVA Studio.

In [ ]:
from langchain_naver import ChatClovaX

llm = ChatClovaX(
    model="HCX-DASH-001",
    temperature=0.5,
    max_tokens=None,
    max_retries=2,
    # clovastudio_api_key="..."  # if you prefer to pass api key in directly instead of using env vars
    # task_id="..."  # if you want to use fine-tuned model
    # service_app=False (default)  # True if using service app
    # include_ai_filters=False (default) ## True if you want to detect inappropriate content
    # other params...
)

## Invocation

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Korean. Translate the user sentence.",
    ),
    ("human", "I love using AI of NAVER CLOVA X."),
]
ai_msg = llm.invoke(messages)
ai_msg

In [ ]:
print(ai_msg.content)

## Chaining

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}. Translate the user sentence.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "input": "I love using AI of NAVER CLOVA X.",
    }
)

## Additional functionalities

### Fine-tuning

You can call fine-tuned CLOVA X models by passing in your corresponding `task_id` parameter. (You don’t need to specify the model_name parameter when calling fine-tuned model.)

You can check `task_id` from corresponding test app/service app details.

In [ ]:
fine_tuned_model = ChatClovaX(
    task_id='abcd123e',
    temperature=0.5,
)

fine_tuned_model(messages)

### Service app

- When applying chatModel to an live/in production service, you should apply and use a service app.
    - https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#서비스앱신청
- For service apps, an API Key is issued and can only be called with that API Key.

In [ ]:
#### Setting environment variables

NCP_CLOVASTUDIO_API_KEY="please input your clova studio api key."
NCP_APIGW_API_KEY="please input your NCP gateway api key."

In [ ]:
llm = ChatClovaX(
    service_app=True, # True if you want to use your service app, default value is False.
    # clovastudio_api_key="..."  # if you prefer to pass api key in directly instead of using env vars
    # apigw_api_key="..."  # if you prefer to pass gateway key in directly instead of using env vars
    model="HCX-DASH-001",
    temperature=0.5,
    max_tokens=None,
    max_retries=2,
    # other params...
)
ai_msg = llm.invoke(messages)

### AI Filter

- AI Filter detects inappropriate output such as profanity from the test app (or service app included) created in Playground and informs the user.
    - https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#AIFilter

In [ ]:
llm = ChatClovaX(
    model="HCX-DASH-001",
    temperature=0.5,
    max_tokens=None,
    max_retries=2,
    include_ai_filters=True, # True if you want to enabled ai filter
    # other params...
)

ai_msg = llm.invoke(messages)

In [ ]:
print(ai_msg.response_metadata['ai_filter'])

## API reference

For detailed documentation of all ChatClovaX features and configurations head to the API reference: https://api.python.langchain.com/en/latest/chat_models/langchain_naver.chat_models.ChatClovaX.html